# Data Analysis

**We are now gonna perform calculations on the aggregated Data to identify informations that will help us solve our work subject. 
To perform the analysis, we will use the SQL language which is a powerfull tool in that purpose.**

## Setting up the environment

We are gonna use the DuckDB database system extension, which will allow us to write our SQL Queries directly in our notebook.
First we need to install it :

In [1]:
!pip install duckdb duckdb-engine jupysql

We now load and connect the SQL database extension :

In [2]:
%load_ext sql
    
%sql duckdb://

Connecting to 'duckdb://'

In [3]:
%config SqlMagic.displaylimit = 50 # before starting we raise the limit of the number of rows displayed after we run our queries to 50, to be sure that every result will be clear to read

## Performing summary statistics

We can start the analysis by testing that the SQL extension is working correctly with a simple query about the number of rides by customer status in the table :

In [4]:
%%sql

SELECT
    member_casual AS user_group, -- we prefer to rename the member_casual column of our file for more clarity
   COUNT(ride_id)
FROM 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Cyclistic_Trips_Final_202502_202601.csv'
GROUP BY member_casual; -- we'll always be interested in grouping the results by user group to compare them, since it's the core of our mission

Running query in 'duckdb://'

user_group,count(ride_id)
member,3481726
casual,1916258


Adding a column to translate this result into a percentage could grant a better insight :

In [15]:
%%sql

SELECT
    member_casual AS user_group,
    COUNT(ride_id) AS number_of_rides, -- we give the alias number_of_rides to our ride_id count for a better understanding of the table
    ROUND(COUNT(ride_id) * 100.0 / SUM(COUNT(ride_id)) OVER(),2) AS Percentage_of_rides -- We use the ROUND function to round the result to decimals, and the OVER function to calculate the percentage for each member category in the total amount 
FROM 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Cyclistic_Trips_Final_202502_202601.csv'
GROUP BY member_casual; 

Running query in 'duckdb://'

user_group,number_of_rides,Percentage_of_rides
member,3481726,64.5
casual,1916258,35.5


The members count for almost the two thirds of the rides done on Cyclistic bikes over the past year. Since we already add the length of the rides in minutes, we can compare these results to the amount of time spent on bike trips for the members and the casual users, in absolute value and in percentage :

In [14]:
%%sql

SELECT
    member_casual AS user_group,
    COUNT(ride_id) AS number_of_rides,
    ROUND(SUM(ride_length_minutes) * 100.0 / SUM(SUM(ride_length_minutes)) OVER(),2) AS percentage_of_minutes_spent_on_bikes, -- again we use round to make the result number clearer, and the OVER function to return the global percentage over the two categories
FROM 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Cyclistic_Trips_Final_202502_202601.csv'
GROUP BY member_casual;


Running query in 'duckdb://'

user_group,number_of_rides,percentage_of_minutes_spent_on_bikes
member,3481726,52.74
casual,1916258,47.26


We notice that even if they only represent 35.5% of the number of bike trips, casual users account for almost half of the time spent on bike trips. The average duration of a trip for casual users must be significately higher. Let's verify the mean duration of trips overall, and by user category :

In [13]:
%%sql 

SELECT
    member_casual AS user_group,
    COUNT(ride_id) AS number_of_rides,
    ROUND(SUM(ride_length_minutes) / COUNT(ride_id) ,2) AS mean_trip_length_minutes, -- we calculate the average length for each category
    ROUND(SUM(SUM(ride_length_minutes)) OVER() / SUM(COUNT(ride_id)) OVER(),2) AS global_length_mean -- and the global length mean with the OVER function
FROM 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Cyclistic_Trips_Final_202502_202601.csv'
GROUP BY member_casual;    

Running query in 'duckdb://'

user_group,number_of_rides,mean_trip_length_minutes,global_length_mean
member,3481726,12.23,14.96
casual,1916258,19.92,14.96


The mean length of a trip is significanly higher for casual members than for annual members. This could mean that members rent a bike mostly for routine or daily trips, whereas casual members would use the service for leisure activities that are less recurrent and predictable.
To look into other potential behavioral differences between user groups, we can run queries to get informations on trips depending of the day of the week they occur :

Let's see the repartition of all trips by day of week :

In [35]:
%%sql

SELECT
    COUNT(ride_id) AS number_of_rides,
    day_of_week,
    day_name
FROM 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Cyclistic_Trips_Final_202502_202601.csv'
GROUP BY day_of_week, day_name
ORDER BY day_of_week ASC;

Running query in 'duckdb://'

number_of_rides,day_of_week,day_name
714563,0,Monday
776734,1,Tuesday
750320,2,Wednesday
807379,3,Thursday
823202,4,Friday
833842,5,Saturday
691944,6,Sunday


The repartition of bike trips over the week is relatively homogeneous. The greatest daily usage of the service occurs on Saturday, whereas the least popular day for riding Cyclistic bicycles is Sunday.

## Looking for trends in the Data

 Let's break down the previous result by showing the number of trips by day of week by user type ; we also want to show the repartition between casual riders and members for each day of the week

In [12]:
%%sql

SELECT
    day_of_week,
    day_name,
    member_casual AS user_group,
    COUNT(ride_id) AS number_of_rides,
    ROUND(COUNT(ride_id)  * 100 / SUM(COUNT(ride_id)) OVER(PARTITION BY day_name),2) AS percentage_of_daily_users -- We use the OVER function with the PARTITION Clause to define the specific window in which we want to compute the SUM to get the percentage by user by day of week
FROM 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Cyclistic_Trips_Final_202502_202601.csv'
GROUP BY member_casual, day_of_week, day_name
ORDER BY day_of_week, member_casual ASC;

Running query in 'duckdb://'

day_of_week,day_name,user_group,number_of_rides,percentage_of_daily_users
0,Monday,casual,219760,30.75
0,Monday,member,494803,69.25
1,Tuesday,casual,218350,28.11
1,Tuesday,member,558384,71.89
2,Wednesday,casual,211743,28.22
2,Wednesday,member,538577,71.78
3,Thursday,casual,247139,30.61
3,Thursday,member,560240,69.39
4,Friday,casual,306348,37.21
4,Friday,member,516854,62.79


We can see two main trends on the table above. First, during the working days, the annual members are overrepresented in the share of riders on every particular day of week except on Fridays, where the relative share of casual members (37.21%) is slightly higher than their share in the total number of trips (35.5%).
Conversely, during the weekend days (Saturday and Sunday), casual members are largely overrepresented in comparison to their share in the annual number of city trips. They account for almost half of the total trips happening on Saturdays (47.41%), and slightly less on Sundays (45.9%).

Let's add a column for the percentage by day of week by user type

In [11]:
%%sql

SELECT
    day_of_week,
    day_name,
    member_casual AS user_group,
    COUNT(ride_id) AS number_of_rides,
    ROUND(COUNT(ride_id)  * 100 / SUM(COUNT(ride_id)) OVER(PARTITION BY member_casual),2) AS percentage_of_users_per_day_of_week -- we use the OVER function with the partition clause again with the member type column this time
FROM 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Cyclistic_Trips_Final_202502_202601.csv'
GROUP BY member_casual, day_of_week, day_name
ORDER BY member_casual, day_of_week  ASC;

Running query in 'duckdb://'

day_of_week,day_name,user_group,number_of_rides,percentage_of_users_per_day_of_week
0,Monday,casual,219760,11.47
1,Tuesday,casual,218350,11.39
2,Wednesday,casual,211743,11.05
3,Thursday,casual,247139,12.9
4,Friday,casual,306348,15.99
5,Saturday,casual,395344,20.63
6,Sunday,casual,317574,16.57
0,Monday,member,494803,14.21
1,Tuesday,member,558384,16.04
2,Wednesday,member,538577,15.47


We notice again the two different trends between casual riders and members : casual riders use the service a lot more the 3 last days of the week, especially on Saturdays and Sundays. Conversely, members are riding bikes proportionnaly more during the working days than during weekends.

Let's observe if there are also differences in the average length of rides depending on the day they occur, grouped by user category :

In [10]:
%%sql

SELECT
    day_of_week,
    day_name,
    member_casual AS user_group,
    COUNT(ride_id) AS number_of_rides,
    ROUND(AVG(ride_length_minutes), 2) AS average_length_by_day -- We use the AVG function to return our average length
FROM 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Cyclistic_Trips_Final_202502_202601.csv'
GROUP BY day_of_week, day_name, member_casual
ORDER BY member_casual, day_of_week  ASC;

Running query in 'duckdb://'

day_of_week,day_name,user_group,number_of_rides,average_length_by_day
0,Monday,casual,219760,19.7
1,Tuesday,casual,218350,17.55
2,Wednesday,casual,211743,16.37
3,Thursday,casual,247139,17.47
4,Friday,casual,306348,19.64
5,Saturday,casual,395344,22.44
6,Sunday,casual,317574,23.1
0,Monday,member,494803,11.83
1,Tuesday,member,558384,11.87
2,Wednesday,member,538577,11.68


Even if the pattern isn't as clear as in the previous queries, we find again a disparity bewteen the two groups : the casual users behavior is more erratic, with important variations in the mean length of rides from one week day to the other (an average length of 16.37 minutes on Wednesdays and 22.44 minutes on Saturdays, representing a 37% gap). On their part, annual members present a very close trip mean length for the working days, and a slightly higher average on weekends.

Let's show the standard deviation of the mean by each group/day to confirm our interpretation :

In [68]:
%%sql

SELECT
    day_of_week,
    day_name,
    member_casual,
    COUNT(ride_id) AS number_of_rides,
    ROUND(AVG(ride_length_minutes), 2) AS average_length_by_day,
    ROUND(STDDEV(ride_length_minutes), 2) AS standard_deviation_per_day -- We just add the standard deviation column with the STDDEV function
FROM 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Cyclistic_Trips_Final_202502_202601.csv'
GROUP BY day_of_week, day_name, member_casual
ORDER BY member_casual, day_of_week  ASC;

Running query in 'duckdb://'

day_of_week,day_name,member_casual,number_of_rides,average_length_by_day,standard_deviation_per_day
0,Monday,casual,219760,19.7,37.82
1,Tuesday,casual,218350,17.55,36.77
2,Wednesday,casual,211743,16.37,32.68
3,Thursday,casual,247139,17.47,36.44
4,Friday,casual,306348,19.64,39.74
5,Saturday,casual,395344,22.44,44.12
6,Sunday,casual,317574,23.1,43.57
0,Monday,member,494803,11.83,20.47
1,Tuesday,member,558384,11.87,20.99
2,Wednesday,member,538577,11.68,19.72


The table above confirm our interpretation, with a strong standard deviation for casual users, signaling unpredictable patterns, and a weaker standard deviation for the members, who have more predictable using habits regarding their Bike share use.

To get more in depth in our study, we can query more specifically the number of rides depending on the time of the day, using the date-time data we have in our aggregated file.
We could break down this data by user category again, to compare the two groups. But since the data will be very long and filled with a lot of numbers, it seems more relevant to pivot it, and to make two separate tables for each customer group, to make the result easier to read. First is the table for the casual users :

In [82]:
%%sql

SELECT *
FROM(SELECT
    day_of_week,
    day_name,
    EXTRACT(HOUR FROM started_at) as start_hour -- we extract the hours from our started_at column, which is formated to DATE TIME format
    FROM 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Cyclistic_Trips_Final_202502_202601.csv'
    WHERE member_casual = 'casual' -- we will focus on individual user groups to make the table easier to interpret. We start with a table for the casual users
)
PIVOT(COUNT(*) FOR start_hour IN (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23)) -- we add the SQL PIVOT syntax to keep our hours as columns and switch the days to rows 
ORDER BY day_of_week ASC;

Running query in 'duckdb://'

day_of_week,day_name,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,Monday,3395,1952,1364,868,792,1538,3765,6707,8914,7582,8261,10838,13156,13752,14605,16407,20531,23998,19644,14321,10022,7703,5935,3710
1,Tuesday,2620,1404,994,524,674,1571,4235,8179,11268,7883,7122,8936,10741,11139,11791,14527,20192,25403,21815,16020,11365,9931,6438,3578
2,Wednesday,2652,1410,907,552,716,1702,4193,8167,10936,7202,6847,8696,10188,9900,10776,13234,18625,24376,20839,15914,11903,10279,7378,4351
3,Thursday,3035,1678,1074,648,747,1611,4231,8378,11145,8708,8427,10548,12888,13045,14476,17109,22136,27991,24048,17563,11972,10231,8928,6522
4,Friday,4774,2960,1829,1016,957,1895,4063,7261,9267,9214,10940,14688,18739,19444,20203,24327,28688,30618,26823,19902,14403,12067,11605,10665
5,Saturday,10249,7253,4757,2484,1352,1220,2503,4718,9148,14941,22193,27873,31349,32285,33544,34300,32697,28566,24414,18774,13894,13179,13188,10463
6,Sunday,9943,6702,4703,2600,1685,1541,2753,4568,7422,12777,18558,23486,26549,27094,27913,28247,26132,22406,19398,14071,10234,7929,6610,4253


And now the table for the annual members :

In [83]:
%%sql

SELECT *
FROM(SELECT
    day_of_week,
    day_name,
    EXTRACT(HOUR FROM started_at) as start_hour
    FROM 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Cyclistic_Trips_Final_202502_202601.csv'
    WHERE member_casual = 'member' -- We now focus on the members habits only
)
PIVOT(COUNT(*) FOR start_hour IN (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23)) -- same syntax as before but for members this time
ORDER BY day_of_week ASC;

Running query in 'duckdb://'

day_of_week,day_name,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,Monday,2586,1485,888,733,1225,5593,16976,33021,40321,22262,15783,18913,22521,21934,23061,31568,50206,62361,45056,30354,20341,14478,8787,4350
1,Tuesday,2291,1238,667,589,1188,6501,20054,41537,52035,25284,16978,19664,23457,22376,22346,33536,57710,70295,50774,33493,22920,17633,10593,5225
2,Wednesday,2627,1303,717,617,1100,6411,19487,40144,48739,25105,16925,19662,23353,21880,22030,31636,53862,65241,47879,32311,22610,17692,11444,5802
3,Thursday,2951,1589,876,634,1176,5570,18269,38042,48897,26515,18660,21602,25505,24135,24536,34959,54780,66897,49671,33431,22513,17711,13269,8052
4,Friday,4427,2477,1378,973,1276,5548,15116,27544,33803,22980,18858,23660,28458,28424,29189,37894,49221,52303,42763,29677,20376,15986,13756,10767
5,Saturday,8459,5685,3538,1885,1142,2109,5169,9271,16577,23099,29103,32629,33175,33071,33342,33333,33522,30759,28325,21671,15730,13664,12933,10307
6,Sunday,7944,5294,3402,2033,1467,1897,4137,6894,11988,18197,24205,28503,29911,29703,29759,30111,29630,27601,25526,19087,13954,10770,7830,4527


By observing the two tables, we notice again different trends bewteen casual users and members : where we see a rather gradual increase of the number of trips throughout the day for casual bikers, we notice a significative peak for the annual members between 7 and 8 hours during the working days. The evolution on weekend days is more linear, casual-like.

One last think we could investigate is the seasonnality of the bikers : do the season impact the number of trips, their length, and how are both groups impacted ?
Let's write a query to verify that :

In [5]:
%%sql

SELECT
    EXTRACT(MONTH FROM started_at) AS month_number,
    MONTHNAME(started_at) AS month_name, -- the MONTHNAME function extracts the name of the month from each cell of our date-time formatted column
    member_casual AS user_group,
    COUNT(ride_id) AS number_of_rides,
    AVG(ride_length_minutes) AS average_length_of_trips_per_month,
    ROUND(COUNT(ride_id) * 100 / SUM(COUNT(ride_id)) OVER(PARTITION BY member_casual),2) AS percentage_of_annual_trips_by_month -- we use the OVER function again, combined with the PARTITION clause to return the percentage of trips per month out of the total of the year for each category 
FROM 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Cyclistic_Trips_Final_202502_202601.csv'
GROUP BY member_casual, month_number, month_name
ORDER BY member_casual, month_number ASC

Running query in 'duckdb://'

month_number,month_name,user_group,number_of_rides,average_length_of_trips_per_month,percentage_of_annual_trips_by_month
1,January,casual,23881,13.028482475608213,1.25
2,February,casual,27003,12.381208013924388,1.41
3,March,casual,82864,17.885376641243486,4.32
4,April,casual,105260,18.655635284058523,5.49
5,May,casual,175655,20.831757479149488,9.17
6,June,casual,278702,21.908670874267173,14.54
7,July,casual,308446,21.438595767168316,16.1
8,August,casual,323533,21.71008166091251,16.88
9,September,casual,254727,19.55189124042604,13.29
10,October,casual,214380,18.094237382218516,11.19


As we can see, more than 60% of all bike trips from casual users occur in the span of only four months, between June and September. Over the same period, bike trips represent a share of less than 50% of the annual rides for the members. Inter-month differences are not as huge for this particular group in comparison to the first one.

Our previous queries have given us a comprehensive overview of the use of Cyclistic Bikes by user group, over different time periods ( day of the week, months, hours of the day). We got relative and absolute data about the number of rides and their length for both groups. Now that we finished our analytic phase, we can export the most interesting results we got in the form of csv files. These files will be later imported in the visualization tool we're gonna use to share our findings. 

## Exporting the Data

First export : a file representing the number of trips per day of week, per hour of the day for both groups of users

In [7]:
%%sql

COPY ( -- We put the data we used in our PIVOT table earlier in a long-data form again, so it will be easier to convert to visualization in our tool
    SELECT
        member_casual AS user_group,
        day_of_week,
        day_name,
        EXTRACT(HOUR from started_at) AS start_hour,
        COUNT(*) AS number_of_rides
    FROM 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Cyclistic_Trips_Final_202502_202601.csv'
    GROUP BY member_casual, day_of_week, day_name, start_hour
    ORDER BY member_casual, day_of_week, start_hour
) TO 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Tableau_Heatmap_Data.csv' (HEADER, DELIMITER ',');
-- we don't forget the HEADER clause to convert the first line of the file to column headers, and delimit it with a comma as it's the standard separator for csv files

Running query in 'duckdb://'

Count


Second export is our seasonnality (monthly) query from above :

In [8]:
%%sql

COPY (
    SELECT
    EXTRACT(MONTH FROM started_at) AS month_number,
    MONTHNAME(started_at) AS month_name,
    member_casual AS user_group,
    COUNT(ride_id) AS number_of_rides,
    AVG(ride_length_minutes) AS average_length_of_trips_per_month,
    ROUND(COUNT(ride_id) * 100 / SUM(COUNT(ride_id)) OVER(PARTITION BY member_casual),2) AS percentage_of_annual_trips_by_month
FROM 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Cyclistic_Trips_Final_202502_202601.csv'
GROUP BY member_casual, month_number, month_name
ORDER BY member_casual, month_number ASC
) TO 'C:/Users/pocit/OneDrive/Documents/formations_reconversion/Google data analyst certificate/Coursera8_Case_Study/Bike_trip_files/All_trips/Tableau_Saisonnality_Data.csv' (HEADER, DELIMITER ',');

Running query in 'duckdb://'

Count


**The files have been successfully generated. Now it's time to visualize our most interesting findings in order to fulfill the objective that was initially given to us. We will use the Tableau tool in this regard**